## Lifetimes

- Lifetimes enforce that a piece of memory for a given reference is still valid.
- Lifetime for a given value (references only?) ends when value is moved or dropped. (So, not necessarily tied to scope.)
- Two references (e.g &x and &y) given a similar lifetime ('a) essentially denote that memory for both is still valid for a given
  area of our code (denoted by 'a). 
  
  

In [9]:
fn moved_b(){
    let a;
    {
        let b = String::from("Hello");
        // Because &T is Copy, assignment of b to a doesn't move
        // contents of b as the following assignment would:
        //
        // a = b
        //
        // Instead, we get, as the name implies, a reference to whatever b
        // is pointing to.
        a = &b
    } // b's memory is dropped here (end of scope). 
    // So, reference to it would be left dangling.
    // println!("{}", a);
}

moved_b()

#### E0106

An example of missing lifetime specifier:

In [13]:
/// No references passed in so ref returned would have to be returned from in function.
/// Also knows that references from inside the function would (probably? definitely?) that all
/// variables created inside the function will be cleaned when the function is finished (end of scope)
/// Applying 'static to it does not help (rust understands that we still return ref to local.)
fn missing_1() -> &i32 {
    let a = 1;
    &a
}

#### E0621

Mismatch in lifetime specifiers.

In [10]:
/// Conditionally return ref.
/// One possible solution: subtyping for lifetimes, i.e place a bound on
/// 'b to tell the compiler to *only* accept lifetimes >= 'b. 
fn condition_ref_return<'a, 'b>(a: &'a i32, b: &'b i32) -> &'a i32 {
    if a > b {
        a
    } else {
        b
    }
}

Error: lifetime mismatch

### Related Errors:

 - [E0106](https://doc.rust-lang.org/error-index.html#E0106)
 - [E0495](https://doc.rust-lang.org/error-index.html#E0495)
 - [E0515](https://doc.rust-lang.org/error-index.html#E0515)
 - [E0621](https://doc.rust-lang.org/error-index.html#E0621)

### Related Videos, Blogs

Videos:

- https://www.youtube.com/watch?v=rAl-9HwD858
- 

### todo?

- static lifetimes
- lifetime ellision